In [ ]:
##### Owner - Rai Rajani 13th May 2022
### This notebook uses Sequence Pattern Matching algorithm of the Customer interaction with benchmark purchase sequences to prediction if the customer will eventually do a transaction or not.
# This algorithm can very well be extended to recommend new items given a customers/visitors interaction . The recommendation can be created by comparing sequence and price of interaction with other item transaction sequences. 
# This code implements the idea for 1 item and can be extended to other items as well

**Data Input and Data Prepration**



In [ ]:
### Import
import pandas as pd
import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls /content/drive/MyDrive/Colab\ Notebooks

category_tree.csv  item_properties_part1.csv  Recommendation_rajani.ipynb
events.csv         item_properties_part2.csv  Video.ipynb


In [ ]:
### reading all the data
category_tree = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/category_tree.csv")
events_log = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/events.csv")
item_properties_part1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/item_properties_part1.csv")
item_properties_part2= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/item_properties_part2.csv")

In [ ]:
## Combining all Items property in 1 Dataframe. Please note the Items property data is not used in the algorithm but the Property like Price with Property ID 790 and Availability can be used to enhance the prediction and recommendation
items_all = pd.concat([item_properties_part1,item_properties_part2])

In [ ]:
items_all.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [ ]:
events_log.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [ ]:
### Date format change and sorting by Date
print(events_log.event.unique())
events_log = events_log.assign(date=pd.Series(datetime.datetime.fromtimestamp(i/1000).date() for i in events_log.timestamp))
events_log = events_log.sort_values('date').reset_index(drop=True)
events_log = events_log[['visitorid','itemid','event', 'date']]
events_log.head(5)

['view' 'addtocart' 'transaction']


,visitorid,itemid,event,date
0,689859,421640,view,2015-05-03
1,935582,203248,view,2015-05-03
2,696326,194830,view,2015-05-03
3,131668,395045,view,2015-05-03
4,595484,129111,view,2015-05-03


In [ ]:
## items all date time change
items_all = items_all.assign(date=pd.Series(datetime.datetime.fromtimestamp(i/1000).date() for i in items_all.timestamp))
items_all = items_all.sort_values('date').reset_index(drop=True)
items_all1 = items_all
items_all1.head(5)


,timestamp,itemid,property,value,date
0,1431226800000,364,6,227961 474154,2015-05-10
1,1431226800000,227976,112,679677,2015-05-10
2,1441508400000,167900,categoryid,438,2015-05-10
3,1431226800000,152517,283,630194 992688 1032603 425005 796651 240388 906...,2015-05-10
4,1431226800000,130187,888,758851,2015-05-10


In [ ]:
#take only those events where transaction was done (Improvement possible - take the addtocard also)
event_visitor_count = events_log[events_log.event == "transaction"]['visitorid'].reset_index()
visitor_transaction= event_visitor_count["visitorid"].unique()
event_onlytrans = events_log[events_log.visitorid.isin(visitor_transaction)]
event_onlytrans.shape

(230678, 4)

In [ ]:
# this is an optional step where you can save the subset event data for future usage
event_onlytrans.to_csv('/content/drive/MyDrive/Colab Notebooks/event_log_transaction.csv')

**Items all and Cat tree Data Exploration**

In [ ]:
#### Explore Category tree # and other items_all col are not useful
##### Exploration shows that there is many to many mapping between itemid -> Category id -> Parent id. Given less information it is difficult to use this data in the algorith
# category_tree.head()
# #take only those items which are available
# cat1 = items_all.loc[(items_all.property == "categoryid") & (items_all.value == "1016")].sort_values('timestamp').head(50)
# print(cat1)
# cat1[['itemid','timestamp']].groupby(['itemid']).count()
# items_all1[items_all1.itemid == 25]
# # pp = items_all_categoryid.groupby(['itemid']).count()
# # print(pp[pp.value>3])
# # print(items_all_categoryid[items_all_categoryid.itemid==3590])
items_all['property'].unique()


### below code tries to get availability and price of each item from Items property file but the availability flag ==1 is very small data set for further use. Price is not used but can be included for across item recommendation
# Get only availability of the items from items all file
items_all_available = items_all1.loc[(items_all1.property == "available")][['itemid','value','date']].drop_duplicates()
items_available = items_all_available[items_all_available.value == '1']
items_available.head()
## Get the price from items 
items_all_price = items_all1.loc[(items_all1.property == "790")][['itemid','value','date']].drop_duplicates()
items_all_price.columns = ['itemid','price','date']
items_all_price["price"] = [i.replace("n","") for i in items_all_price["price"]]
items_all_price
items_final = items_all_price.merge(items_available, on=['itemid','date'],how='left')

[array(['6', '112', 'categoryid', ..., '744', '722', '769'], dtype=object)]

In [ ]:
#Again a good habit to save the merged dataset for future usage
items_final.to_csv('/content/drive/MyDrive/Colab Notebooks/items.final.csv')

**Let's start using Event log data to extract Baseline sequence and Transaction sequence to build Sequnce similarity matching for prediction**

In [2]:
### save the light file and read again
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
import pandas as pd

In [141]:
events_log = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/event_log_transaction.csv")
item_all = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/items.final.csv")

In [142]:
events_log.head()

,Unnamed: 0,visitorid,itemid,event,date
0,18,163561,422425,view,2015-05-03
1,36,163561,313858,view,2015-05-03
2,38,163561,274798,view,2015-05-03
3,86,1378125,423944,view,2015-05-03
4,114,935325,39440,view,2015-05-03


In [143]:
item_all.head()

,Unnamed: 0,itemid,price,date,value
0,0,149578,383280.0,2015-05-10,1.0
1,1,343393,79440.0,2015-05-10,1.0
2,2,207935,78480.0,2015-05-10,NaN
3,3,267535,36240.0,2015-05-10,1.0
4,4,195857,7680.0,2015-05-10,NaN


In [144]:
events_log = events_log.drop(columns = 'Unnamed: 0')
item_all = item_all.drop(columns = 'Unnamed: 0')
item_all.columns = ['itemid','price','date','availability']

In [145]:
# Let's use train set by taking first 2 months of the data. You can do a 70/30 split for actual implementation
events_log = events_log.sort_values(['visitorid','itemid','date'])
print(events_log.shape)
events_log_train = events_log[events_log.date < "2015-07-31"]
print(events_log_train.shape)


(230678, 4)
(162421, 4)


In [146]:
### this cell is commented because the overall between the items file and event file for available items is very less

# item_all1 = item_all[['itemid','date','price']].groupby(['itemid','date']).agg({'price':'mean'})
# item_all1 = item_all.reset_index(inplace = False)
# item_all = item_all1.drop(columns = 'index')
# events_log = events_log.sort_values(['visitorid','itemid','date'])
# print(events_log.shape)
# events_log_available = events_log.merge(item_all,on=['itemid','date'],how='left')
# print(events_log_available.shape)
# events_log_available = events_log_available.dropna()
# print(events_log_available.shape)

In [147]:
### This code creates sequence of event transactions for all the interactions where purchase was finally made
visitor_item_seq= pd.DataFrame()
seq = 0
for i in events_log_train.visitorid.unique():
  filter_visitor = events_log_train[events_log_train.visitorid == i]
  # print(filter_visitor.shape)
  for j in filter_visitor.itemid.unique():
    filter_visitor_item = filter_visitor[filter_visitor.itemid == j]
    if filter_visitor_item.shape[0] >1:
      if visitor_item_seq.shape[0]>0:
        if (visitor_item_seq.event.iloc[-1] != "transaction"):
          seq =seq+1
    # print(filter_visitor_item.shape)
      for jj in filter_visitor_item.index:
        if (filter_visitor_item.loc[jj]['event']  != "transaction"):  
          kk = filter_visitor_item.loc[jj].copy()
          kk['seq'] = seq
          visitor_item_seq = visitor_item_seq.append(kk)
        else:
          kk = filter_visitor_item.loc[jj].copy()
          kk['seq'] = seq
          visitor_item_seq = visitor_item_seq.append(kk)
          seq = seq+1


In [148]:
visitor_item_seq.head(50)
## best Practice to save the sequence file for future use
visitor_item_seq.to_csv('/content/drive/MyDrive/Colab Notebooks/visitor_item_seq.csv')

,visitorid,itemid,event,date,seq
130483,172.0,10034.0,view,2015-07-14,0.0
130484,172.0,10034.0,view,2015-07-14,0.0
130883,172.0,10034.0,view,2015-07-14,0.0
133736,172.0,10034.0,view,2015-07-15,0.0
131000,172.0,292240.0,view,2015-07-14,1.0
133215,172.0,292240.0,view,2015-07-15,1.0
133636,172.0,292240.0,view,2015-07-15,1.0
133711,172.0,292240.0,view,2015-07-15,1.0
130999,172.0,464731.0,view,2015-07-14,2.0
131025,172.0,464731.0,view,2015-07-14,2.0


In [149]:
### we are ready with the sequence now let's build benchmark sequence for 1 item id to start with
# we will use these benchmark sequence to predict for test data Purchase of the same item

visitor_item_seq=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/visitor_item_seq.csv')

In [150]:
visitor_item_seq.itemid.value_counts().sort_values()
# highest count item 119736

296561.0      2
267663.0      2
121311.0      2
354542.0      2
58217.0       2
           ... 
312728.0    247
445351.0    267
306289.0    308
461686.0    372
119736.0    439
Name: itemid, Length: 14995, dtype: int64

In [151]:
# filter the event data for 1 item ID and build its benchmark sequence
visitor_item_seq_item1 = visitor_item_seq[visitor_item_seq.itemid == 119736]
visitor_item_seq_item1.shape

(439, 6)

In [152]:
# extract seq where transaction has occured at the end 

seq_transaction = visitor_item_seq_item1[visitor_item_seq_item1.event == "transaction"]['seq'].unique()
seq_transaction = set(seq_transaction)

In [153]:
visitor_item_seq_item1_onlytrans = visitor_item_seq_item1[visitor_item_seq_item1.seq.isin(seq_transaction)]
visitor_item_seq_item1_onlytrans.shape
visitor_item_seq_item1_event = visitor_item_seq_item1_onlytrans[['event','seq']]
import numpy as np
seq_list = visitor_item_seq_item1_event.groupby(['seq']).agg(lambda x: x.astype('str').replace(0, np.nan).dropna().tolist()).reset_index(inplace=False)
seq_only_seq = seq_list['event'].drop_duplicates()
seq_list
### you can very well select benchmark sequence which are most occuring and has a sepecific lenght. A sequence which has only ['transaction'] might not be useful but can also say that its mostly commanly purchased without any view. 

,seq,event
0,1190.0,[transaction]
1,2163.0,[transaction]
2,2164.0,[transaction]
3,2165.0,"[view, transaction]"
4,2166.0,[transaction]
...,...,...
58,35925.0,"[view, view, view, transaction]"
59,35926.0,"[view, addtocart, view, view, transaction]"
60,38214.0,"[view, transaction]"
61,39211.0,"[view, view, view, view, view, view, view, vie..."


In [154]:
## Now we have our Benchmark sequence ready. Let's use a test data for prediction
### Let's look at the test data transactions and remove the transaction event to prediction matching of sequence
events_log_test = events_log[events_log.date > "2015-07-31"]
print(events_log_test.shape)
event_log_test_item1 = events_log_test[events_log_test.itemid == 119736]

(66544, 4)


In [155]:
## Create transaction sequence for test data . we will remove the final transaction event afterwards. This is to ensure reusability of previous code. Also, this should be used at a realtime based else the algorithm 
#will not be able to differentiate between multiple sequences. # you can very well add a perticulate filter to capture a sequence
visitor_item_seq= pd.DataFrame()
seq = 0
for i in event_log_test_item1.visitorid.unique():
  filter_visitor = event_log_test_item1[event_log_test_item1.visitorid == i]
  # print(filter_visitor.shape)
  for j in filter_visitor.itemid.unique():
    filter_visitor_item = filter_visitor[filter_visitor.itemid == j]
    if filter_visitor_item.shape[0] >1:
      if visitor_item_seq.shape[0]>0:
        if (visitor_item_seq.event.iloc[-1] != "transaction"):
          seq =seq+1
    # print(filter_visitor_item.shape)
      for jj in filter_visitor_item.index:
        if (filter_visitor_item.loc[jj]['event']  != "transaction"):  
          kk = filter_visitor_item.loc[jj].copy()
          kk['seq'] = seq
          visitor_item_seq = visitor_item_seq.append(kk)
        else:
          kk = filter_visitor_item.loc[jj].copy()
          kk['seq'] = seq
          visitor_item_seq = visitor_item_seq.append(kk)
          seq = seq+1


In [156]:
# here we remove the transaction event because this is what we need to predict
remove_visitor_item_seq = visitor_item_seq[visitor_item_seq.event != "transaction"]


In [157]:
remove_visitor_item_seq

,visitorid,itemid,event,date,seq
166833,152963.0,119736.0,view,2015-08-03,0.0
168474,152963.0,119736.0,view,2015-08-04,0.0
169637,152963.0,119736.0,view,2015-08-04,1.0
169655,152963.0,119736.0,view,2015-08-04,1.0
172136,152963.0,119736.0,view,2015-08-06,1.0
...,...,...,...,...,...
164552,1293358.0,119736.0,view,2015-08-01,45.0
164751,1293358.0,119736.0,view,2015-08-01,45.0
164975,1293358.0,119736.0,view,2015-08-01,45.0
165645,1293358.0,119736.0,view,2015-08-02,45.0


In [158]:
## This is the sequence similarity function which compares sequence of 2 given lists and provides a similarity score/ratio
import difflib
def similar_seq(a,b):
  mbs = difflib.SequenceMatcher(None,a, b)
  return mbs.ratio()


In [159]:
### Now for each customer and given Itemid we get similarity of benchmark sequence and the current event sequence
customer_buy_prediction_fn=pd.DataFrame()
customer_buy_prediction=pd.DataFrame()
for cust in remove_visitor_item_seq.visitorid.unique():
  customer_prediction1 = remove_visitor_item_seq[remove_visitor_item_seq.visitorid == cust]
  for dd in customer_prediction1.date.unique():
    customer_prediction2 = customer_prediction1[customer_prediction1.date == dd]
    customer_prediction_seq = customer_prediction2[['event','seq']]
    customer_seq_list = customer_prediction_seq.groupby(['seq']).agg(lambda x: x.replace(0, np.nan).dropna().tolist()).reset_index(inplace=False)
    # print(customer_seq_list['event'])
    for ev in seq_only_seq:
      # print(ev)
      # print(customer_seq_list['event'][0])
      ss = similar_seq(ev, customer_seq_list['event'][0])
      # print(cust)
      customer_buy_prediction = customer_prediction2.iloc[-1][['visitorid','itemid']]
      customer_buy_prediction['visitorid'] = cust
      customer_buy_prediction['date'] = dd
      customer_buy_prediction['itemid'] = 119736.0
      customer_buy_prediction['Buysimi'] = np.where(ss >0.6, 1,0) ### This cutoff can be refined based on itemid and usecase
      customer_buy_prediction_fn = customer_buy_prediction_fn.append(customer_buy_prediction)


In [162]:
customer_buy_prediction_fn

,visitorid,itemid,date,Buysimi
166833,152963.0,119736.0,2015-08-03,0
166833,152963.0,119736.0,2015-08-03,1
166833,152963.0,119736.0,2015-08-03,0
166833,152963.0,119736.0,2015-08-03,0
166833,152963.0,119736.0,2015-08-03,0
...,...,...,...,...
165768,1293358.0,119736.0,2015-08-02,0
165768,1293358.0,119736.0,2015-08-02,0
165768,1293358.0,119736.0,2015-08-02,0
165768,1293358.0,119736.0,2015-08-02,0


In [163]:
#Given the similarity is calculated for each sequence of customer-date interaction, we will group by at customer-date level to see if at-least 1 sequence predicts buying

customer_buy_prediction_buyflag = customer_buy_prediction_fn.groupby(['visitorid','itemid','date'])['Buysimi'].sum().reset_index(inplace=False)
customer_buy_prediction_buyflag['Prediction_Buy_Flag'] = np.where(customer_buy_prediction_buyflag.Buysimi>0,"Will Buy","Will Not Buy")

In [164]:
# Algorithm Accuracy Check on test Data
# Get the transaction data from test data which was removed for prediction earlier
add_visitor_item_seq = visitor_item_seq[visitor_item_seq.event == "transaction"]
# Merge the predictions
accuracy_df = add_visitor_item_seq.merge(customer_buy_prediction_buyflag,on=['visitorid','itemid','date'])
# Count how many predictions are capture in the transaction data. Please note - the item id is selected without much analysis but still shows good results. Please select a better Itemid for checking the effectiveness of the algorithm
accuracy_df.Prediction_Buy_Flag.value_counts()

Will Buy    9
Name: Prediction_Buy_Flag, dtype: int64

In [165]:
accuracy_df

,visitorid,itemid,event,date,seq,Buysimi,Prediction_Buy_Flag
0,152963.0,119736.0,transaction,2015-08-04,0.0,1,Will Buy
1,530559.0,119736.0,transaction,2015-08-02,21.0,1,Will Buy
2,530559.0,119736.0,transaction,2015-08-19,22.0,2,Will Buy
3,530559.0,119736.0,transaction,2015-08-30,23.0,2,Will Buy
4,530559.0,119736.0,transaction,2015-09-04,24.0,3,Will Buy
5,645525.0,119736.0,transaction,2015-08-29,29.0,1,Will Buy
6,645525.0,119736.0,transaction,2015-08-29,30.0,1,Will Buy
7,686012.0,119736.0,transaction,2015-08-12,32.0,1,Will Buy
8,1146790.0,119736.0,transaction,2015-08-03,42.0,1,Will Buy
